In [ ]:
# !pip install object_detection

In [ ]:
import pandas as pd
import tensorflow as tf
import os
import io
from PIL import Image
from object_detection.utils import dataset_util
from tqdm import tqdm

In [ ]:
df=pd.read_csv('/content/Reduced_Emotic_Train.csv')

In [ ]:
df

,File_Name,File_Path,Img_Height,Img_Width,X_Min,X_Max,Y_Min,Y_Max,Affection,Anger,...,Pain,Peace,Pleasure,Sadness,Sensitivity,Suffering,Surprise,Sympathy,Yearning,Label_Names
0,00qbvchuemlwx6muhk.jpg,/content/drive/MyDrive/EMOTIC_YO/emotic/emodb_...,333,500,201,310,104,235,0,0,...,0,0,0,0,0,0,0,0,0,Anticipation
1,041tn9hyq4gw9hsj4x.jpg,/content/drive/MyDrive/EMOTIC_YO/emotic/emodb_...,1511,1984,530,1601,212,1509,0,0,...,0,1,0,0,0,0,0,0,0,Peace
2,0847gkjc02gfaqck3p.jpg,/content/drive/MyDrive/EMOTIC_YO/emotic/emodb_...,360,640,190,504,24,360,0,0,...,0,0,0,1,0,0,0,0,0,Sadness
3,0apauiwt5nd0id0qx2.jpg,/content/drive/MyDrive/EMOTIC_YO/emotic/emodb_...,350,588,263,388,49,351,1,0,...,0,0,0,0,0,0,0,0,0,Affection
4,0hvf7s67453i8zu8gi.jpg,/content/drive/MyDrive/EMOTIC_YO/emotic/emodb_...,240,159,-1,124,41,241,1,0,...,0,0,0,0,0,0,0,0,0,Affection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5971,frame_lxbgorw85y2nkvsw.jpg,/content/drive/MyDrive/EMOTIC_YO/emotic/frames...,434,634,280,339,172,317,0,0,...,0,0,0,0,0,0,0,0,0,Happiness
5972,frame_lyfzgfu3oblk8kjj.jpg,/content/drive/MyDrive/EMOTIC_YO/emotic/frames...,480,640,214,258,139,258,0,0,...,0,0,0,0,0,0,0,0,0,Happiness
5973,frame_lz4goqbyld98peux.jpg,/content/drive/MyDrive/EMOTIC_YO/emotic/frames...,320,450,1,116,140,303,0,0,...,0,0,0,0,0,0,0,0,0,Confidence
5974,frame_m1uniynx9tz03y0v.jpg,/content/drive/MyDrive/EMOTIC_YO/emotic/frames...,3240,4320,2431,2694,903,1761,0,0,...,0,0,0,0,0,0,0,0,0,Engagement


In [ ]:
imgname = df["File_Name"].tolist()
filepath = df["File_Path"].tolist()
iheight = df["Img_Height"].tolist()
iwidth = df["Img_Width"].tolist()
ixmin = df["X_Min"].tolist()
ixmax = df["X_Max"].tolist()
iymin = df["Y_Min"].tolist()
iymax = df["Y_Max"].tolist()
lname = df['Label_Names'].tolist()

In [ ]:
print(filepath[0])

/content/drive/MyDrive/EMOTIC_YO/emotic/emodb_small/images/00qbvchuemlwx6muhk.jpg


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dir_path = '/content/drive/MyDrive/EMOTIC_YO/emotic'
cat = ['Affection','Anger','Annoyance','Anticipation','Aversion','Confidence','Disapproval','Disconnection','Disquietment','Doubt/Confusion','Embarrassment','Engagement','Esteem','Excitement','Fatigue','Fear','Happiness','Pain','Peace','Pleasure','Sadness','Sensitivity','Suffering','Surprise','Sympathy']

In [ ]:
def image_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()])
    )

def bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))


def float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def int64_feature_list(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def float_feature_list(value):
    """Returns a list of float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

In [ ]:
cat.index('Affection')

0

In [ ]:
def get_labels_(example,lname):
  class_text = bytes(lname[example], 'utf-8')
  classes = cat.index(lname[example])
  return classes,class_text

In [ ]:
def load_img_path_(info_dict):
  img_name = str(info_dict[b'filename'])[2:-1]
  folder_name = str(info_dict[b'folder'])[2:-1]
  img_path = dir_path + folder_name + '/' + img_name
  return img_path

In [ ]:
def load_imgpath(example,filepath):
  imgpath = filepath[example]
  return imgpath

In [ ]:
def get_coord(example,iheight,iwidth,ixmin,ixmax,iymin,iymax):
  x1 = ixmin[example]
  x2 = ixmax[example]
  y1 = iymin[example]
  y2 = iymax[example]
  w = iwidth[example]
  h = iheight[example]
  return [w,h, x1/w, y1/h, x2/w, y2/h]

In [ ]:
def create_example(path, example):
  with tf.io.gfile.GFile(os.path.join('{}'.format(path)), 'rb') as fid:
    encoded_jpg = fid.read()
  encoded_jpg_io = io.BytesIO(encoded_jpg)
  image = Image.open(encoded_jpg_io)
  width,height , xmins, ymins , xmaxs , ymaxs = get_coord(example,iheight,iwidth,ixmin,ixmax,iymin,iymax)
  classes,classes_text = get_labels_(example,lname)
  filename = bytes(imgname[example],'utf-8')
  image_format = b'jpg'

  feature={
          'image/height': dataset_util.int64_feature(height),
          'image/width': dataset_util.int64_feature(width),
          'image/filename': dataset_util.bytes_feature(filename),
          'image/source_id': dataset_util.bytes_feature(filename),
          'image/encoded': dataset_util.bytes_feature(encoded_jpg),
          'image/format': dataset_util.bytes_feature(image_format),
          'image/object/bbox/xmin': float_feature(xmins),
          'image/object/bbox/xmax': float_feature(xmaxs),
          'image/object/bbox/ymin': float_feature(ymins),
          'image/object/bbox/ymax': float_feature(ymaxs),
          'image/object/class/text': dataset_util.bytes_feature(classes_text),
          'image/object/class/label': dataset_util.int64_feature(classes)
          }

  return tf.train.Example(features=tf.train.Features(feature=feature))

In [ ]:
create_example(filepath[0], 0)

features {
  feature {
    key: "image/encoded"
    value {
      bytes_list {
        value: "\377\330\377\340\000\020JFIF\000\001\001\000\000\001\000\001\000\000\377\333\000C\000\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\377\333\000C\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\377\300\000\021\010\001M\001\364\003\001\"\000\002\021\001\003\021\001\377\304\000\037\000\000\001\005\001\001\001\001\001\001\000\000\000\000\000\000\000\000\001\002\003\004\005\006\007\010\t\n\013\377\304\000\265\020\000\002\001\003\003\002\004\003\005\005\004\004\000\000\001}\001\002\00

In [ ]:
# with tf.io.TFRecordWriter("csv.tfrecords") as writer:
#     for row in csv:
#         features, label = row[:-1], row[-1]
#         example = tf.train.Example()
#         example.features.feature["features"].float_list.value.extend(features)
#         writer.write(example.SerializeToString())

In [ ]:
len(df)

5976

In [ ]:
df.iloc[0]

File_Name                                     00qbvchuemlwx6muhk.jpg
File_Path          /content/drive/MyDrive/EMOTIC_YO/emotic/emodb_...
Img_Height                                                       333
Img_Width                                                        500
X_Min                                                            201
X_Max                                                            310
Y_Min                                                            104
Y_Max                                                            235
Affection                                                          0
Anger                                                              0
Annoyance                                                          0
Anticipation                                                       1
Aversion                                                           0
Confidence                                                         0
Disapproval                       

In [ ]:
# tfrecords_dir = "/content/drive/MyDrive/EMOTIC_YO/training"
tfrecords_dir = "/content/training"

num_samples = 1
num_tfrecords = len(df) // num_samples
if len(df) % num_samples:
    num_tfrecords += 1  # add one record if there are any remaining samples

if not os.path.exists(tfrecords_dir):
    os.makedirs(tfrecords_dir)

In [ ]:
# lis = []

In [ ]:
with tf.io.TFRecordWriter(tfrecords_dir + "/" + "train.tfrecord") as writer:
  for tfrec_num in tqdm(range(len(df))):
    sample = tfrec_num
    # print(sample)
    example = create_example(filepath[sample], sample)
    # lis.append(example)
    writer.write(example.SerializeToString())

print('Successfully created the TFRecords training')

100%|██████████| 2/2 [00:00<00:00, 195.27it/s]

Successfully created the TFRecords training


In [ ]:
# file1 = open('exampletxt', 'w')
# file1.writelines(str(lis))
# file1.close()